In [26]:
import csv, sys
import os, glob
from string import ascii_lowercase ,digits
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd
from nltk.probability import FreqDist



output_path = '/home/romina/Documents/work/Speeches_mining/outputs'
input_path = '/home/romina/Documents/work/speeches-segmented/greg-tables'

In [27]:
os.chdir(output_path)
output = open('speeches_2005_12_n_100.csv', 'r')
data = pd.read_csv(output, encoding='utf-8')
text = pd.DataFrame(data[['text']])
data = list()
for index, content in text.iterrows():
    data.append(content['text'])

In [28]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)
    ##setting up and extending stopwords
stop = stopwords.words('english')
stop.extend(
    ['nevertheless', 'would', 'nether''the', 'in', 'may', 'also', 'zero', 'one', 'two', 'three', 'four', 'five',
     'six', 'seven', 'eight', 'nine', 'ten', 'quot', 'across', 'among', 'beside', 'however', 'yet',
     'within'] + list(ascii_lowercase))

#for removing digits
translator = str.maketrans('', '', digits)


tokens = []
for statement in data:
    statement_removed_digits=statement.translate(translator)
    words = word_tokenize(statement_removed_digits.lower())
    words_clean = []
    for word in words:
        if word not in stop:
            words_clean.append(stemmer.stem(word))
    tokens.append(words_clean)

In [29]:
n=3
all_trigrams=list()
text_trigram=list()
for token_group in tokens:
    trigram_group = list(nltk.ngrams(token_group, n))
    text_trigram.append(trigram_group)
    for trigram in trigram_group:
        all_trigrams.append(trigram)


In [30]:
total=len(all_trigrams)
cutoff = 0.00003 * total
fdist = FreqDist(all_trigrams)
selected_trigrams={k: v for k, v in fdist.items() if v > cutoff}
selected_trigrams_count=list(selected_trigrams.values())
selected_trigrams=list(selected_trigrams.keys())


In [31]:
binary_matrix = np.zeros(shape=(len(text_trigram), len(selected_trigrams)))
frequency_matrix = np.zeros(shape=(len(text_trigram), len(selected_trigrams)))

for index in range(0, len(text_trigram)):
    for token in text_trigram[index]:
        if token in selected_trigrams:
            binary_matrix[index, selected_trigrams.index(token)] = 1
            frequency_matrix[index, selected_trigrams.index(token)]=text_trigram[index].count(token)


np.savetxt("speeches_2005_12_binary_n_100.csv", binary_matrix.astype(int),fmt='%i', delimiter=",")
np.savetxt("speeches_2005_12_frequency_n_100.csv",frequency_matrix.astype(int),fmt='%i', delimiter=",")

In [61]:
# from numpy import genfromtxt
# os.chdir(output_path)
# text1=genfromtxt('binary_matrix_1.csv', delimiter=',')
# text2=genfromtxt('binary_matrix.csv', delimiter=',')
# print(text1.shape)
# for i in range(1008):
#     if text1.item(0,i)!=text2.item(0,i):
#         print(i,"  ",text1.item(0,i),"  ",text2.item(0,i))
# np.array_equal(text1,text2)

(2, 1008)


True

In [32]:
from statistics import mean, pstdev
import matplotlib.pyplot as plt
mu=mean(selected_trigrams_count)
st=pstdev(selected_trigrams_count,mu)
# plt.hist(selected_trigrams_count, bins=np.arange(min(selected_trigrams_count),max(selected_trigrams_count)+1), align='left')
# plt.show()

In [33]:
f = open('trigrams_position_n_100.csv', 'w')
w = csv.writer(f)
w.writerow(['stats:',mu,st])
w.writerow(['position', 'trigrams','count'])
for index in range(len(selected_trigrams)):
    w.writerow([index,selected_trigrams[index],selected_trigrams_count[index] ])
